# The siibra tool suite – interfaces to a multilevel brain atlas spanning scales and modalities

Studying the human brain requires to capture its structural and functional organization in a common spatial framework. siibra is a tool suite that implements a multilevel atlas of the human brain by providing access to reference templates at different spatial scales, complementary parcellation maps, and multimodal data features. 

For interactive and explorative use, the tool suite includes a web-based 3D viewer hosted [on EBRAINS](https://atlases.ebrains.eu/viewer). [siibra-python](https://siibra-python.readthedocs.io) is a software library for using the framework in computational workflows, and provides good compatibility with established (neuro)data science tools such as [nibabel](https://nipy.org/nibabel/) and [pandas](https://pandas.pydata.org).

this notebook walks you through some examples that are intended for presenting siibra at the OHBM 2023 conference. It uses a recent develoment snapshot of siibra.

In [ ]:
!pip install -U --user siibra

In [ ]:
import siibra as sb
from packaging.version import Version
assert Version(sb.__version__) >= Version('1.0a22')
from nilearn import plotting
import matplotlib.pyplot as plt
%matplotlib notebook

# Load example image with regions of interest

The example starts with a NIfTI image containing some regions of interesta in MNI space, which we are going to assign to cytoarchitectonic brain regions, and use for multimodel data queries. Here we just load a predefined NIfTI file, but in general you might use a thresholded functional activation map or similar signal here.

In [ ]:
img = sb.volumes.from_file(
   "https://github.com/FZJ-INM1-BDA/siibra-tutorials/raw/main/workshops/ohbm-2023-example-input.nii.gz",
    name="Example input", 
    space="mni152",
)

# display the image
plotting.plot_stat_map(img.fetch(), cmap='viridis', colorbar=False, draw_cross=False)

# Assign activations to cytoarchitectonic brain regions

We request the Julich-Brain probabilistic cytoarchitectonic maps, and perform an assignment of the image signal to the cytoarchitectonic structures. Siibra will automatically identify separated structures in the image. The result is pandas dataframe with regions assigned to each structure identified in the input image, and several scores for the assignment. 

In [ ]:
# retrieve Julich-Brain probabilistic cytoarchitectonic maps
julich_pmaps = sb.get_map(
    sb.parcellations.get('julich 2.9'),
    sb.spaces.get('mni 152'),
    sb.MapType.STATISTICAL
)

# assign the input image from above to the cytoarchitectonic maps.
# This results in a pandas DataFrame, which we sort by correlation.
matches = julich_pmaps.assign(img).sort_values('correlation', ascending=False)

# For this example, we work only with a few of the assigned structures.
# We filter by strong correlation and containedness scores.
matches = matches[(matches.correlation > 0.3) & (matches['map containedness'] < 0.25)]

# display some columns of the filtered table
matches[['input structure', 'region', 'correlation']].round(2)

In [ ]:
matches

Show these filtered matched brain regions in 3D.

In [ ]:
fig, axs = plt.subplots(1, len(matches), figsize=(8, 4))
for i, match in enumerate(matches.itertuples()):
    plotting.plot_stat_map(
        julich_pmaps.fetch(region=match.region),
        axes=axs[i], colorbar=False, display_mode='z',
        cut_coords=[int(match.region.compute_centroids('mni152')[0][2])],
        title=match.region.name
    )

# Query multimodal regional features

Retrieving regional data features is as simple as specifying an atlas concept - such as a region, parcellation or space -  and a feature modality. So in order to find receptor density fingerprints in the left primary visual cortex, we could do:

In [ ]:
# retrieve the features for v1 left
features = sb.features.get(
    sb.get_region('julich 2.9', 'v1 left'),
    sb.features.molecular.ReceptorDensityFingerprint,
)

# display the data array of the first of the returned features
features[0].data.T

We use the same mechanism now to build a table of three different feature modalities measured in the two selected brain regions. We choose receptor densities and cell densities, and complement them with connectivity profiles below. The connectivity uses a slightly different query, since the connectivity matrix is linked to the parcellation object, not a single region.

In [ ]:
# retrieve the features for v1 left
features = sb.features.get(
    sb.get_region('julich 2.9', 'v1 left'),
    sb.features.molecular.GeneExpressions,
    gene=['GABBR1', 'GABBR2']
)

# display the data array of the first of the returned features
features[0].data

In [ ]:
# show their locations
plotting.plot_markers(node_coords = list(features[0].data.mni_xyz), node_values=features[0].data['sample'])

In [ ]:
# create the plot
fig, axs = plt.subplots(
    3,
    len(matches),
    sharey='row', figsize=(9,9)
)

# row 1 - receptor densities for different receptor types
for i, m in enumerate(matches.itertuples()):
    features = sb.features.get(
        m.region, 
        sb.features.molecular.ReceptorDensityFingerprint
    )
    features[0].plot(ax=axs[0, i])

# row 2 - cell densities per cortical layer
for i, m in enumerate(matches.itertuples()):
    features = sb.features.get(
        m.region, 
        sb.features.cellular.LayerwiseCellDensity
    )
    features[0].plot(ax=axs[1, i])
        
# row 3 - gene expression
for i, m in enumerate(matches.itertuples()):
    features = sb.features.get(
        m.region, 
        sb.features.molecular.GeneExpressions,
        gene=['MAOA', 'TAC1', 'GABBR1', 'GABBR2']
    )
    features[0].data.boxplot(column='zscore', by='gene', ax=axs[2, i])

# optimize plot layout
plt.tight_layout()

# Extract BigBrain image data

Finally, we sample 3D chunks of the BigBrain volume located in the identified brain regions.

In [ ]:
# access the BigBrain reference template
bigbrain = sb.get_template('bigbrain')

# fetch the whole-brain volume at reduced resolution
bigbrain_volume = bigbrain.fetch(resolution_mm=.64)

In [ ]:
# prepare the plot
f, axs = plt.subplots(2, len(matches), figsize=(9, 7))
plot_kwargs = {
    "bg_img": None,
    "cmap": 'gray',
    "colorbar": False, 
    "draw_cross": False,
    "annotate": False, 
    'vmin': 0,
    'vmax': 255
}

# for each matched brain region, sample a random 3D location in MNI space,
# warp it to bigbrain space, and fetch a 3D chunk of 3mm sidelength
# from the full resolution Big Brain (20 micron) at this position.
for i, match in enumerate(matches.itertuples()):

    point = julich_pmaps.sample_locations(match.region, 1)[0].warp('bigbrain')
    view = plotting.plot_img(bigbrain_volume, axes=axs[0, i], cut_coords=tuple(point), **plot_kwargs)
    view.add_markers([tuple(point)])

    voi = point.get_enclosing_cube(width_mm=3)
    chunk = bigbrain.fetch(voi=voi, resolution_mm=0.02)
    plotting.plot_img(chunk, axes=axs[1, i], **plot_kwargs)

plt.tight_layout()